In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
# Initialize Inception model
import torchvision.models as models
import numpy as np

In [2]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 36.6 MB/s eta 0:00:00


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
"""
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 299x299
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
"""
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet50
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [5]:
# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)

100%|██████████| 170498071/170498071 [00:02<00:00, 73494983.47it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
# Initialize ResNet50 model
"""
import torchvision.models as models
model = models.inception_v3(pretrained=True)
num_classes = 10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
"""

# Initialize ResNet50 model
import torchvision.models as models
# Load pre-trained DenseNet model
model = models.densenet121(pretrained=True)

# Modify the final layer to have 10 outputs (for CIFAR-10 classes)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 10)

for param in model.parameters():
    param.requires_grad = True


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
"""
import torch
import pennylane as qml

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def quantum_net(q_input_features, q_weights_flat):
    # Reshape weights
    q_weights = q_weights_flat.reshape((4, 6))  # Assuming 6 parameters per qubit

    # Quantum circuit
    for i in range(4):
        qml.Rot(q_input_features[i][0], q_input_features[i][1], q_input_features[i][2], wires=i)
        qml.RX(q_weights[i][0], wires=i)
        qml.RY(q_weights[i][1], wires=i)
        qml.RZ(q_weights[i][2], wires=i)
        qml.RX(q_weights[i][3], wires=i)  # Additional gate
        qml.RY(q_weights[i][4], wires=i)  # Additional gate
        qml.RZ(q_weights[i][5], wires=i)  # Additional gate

    return [qml.expval(qml.PauliZ(i)) for i in range(4)]
"""

import torch
import torch.nn as nn
import pennylane as qml

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def quantum_net(q_input_features, q_weights_flat):
    # Reshape weights
    q_weights = q_weights_flat.reshape((4, 6))  # Assuming 6 parameters per qubit

    # Quantum circuit
    for i in range(4):
        qml.RX(q_input_features[i][0], wires=i)
        qml.RY(q_input_features[i][1], wires=i)
        qml.RZ(q_input_features[i][2], wires=i)
        qml.CNOT(wires=[i, (i + 1) % 4])  # Entangling gate

    return [qml.expval(qml.PauliZ(i)) for i in range(4)]


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
"""
# Inside the QuantumNetLayer class
class QuantumNetLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.q_weights_flat = nn.Parameter(torch.randn(4, 6).to(device))  # Define trainable quantum weights
        self.fc = nn.Linear(4, output_size).to(device)  # Adjust the input size to match the output of the quantum circuit

    def forward(self, x):
        q_features = quantum_net(torch.flatten(x, start_dim=1), self.q_weights_flat)
        q_features_tensor = torch.tensor(q_features, dtype=torch.float32).unsqueeze(0).to(device)
        x = self.fc(q_features_tensor)
        return x
"""

class QuantumNetLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.q_weights_flat = nn.Parameter(torch.randn(4, 6).to(device))  # Define trainable quantum weights
        self.fc = nn.Linear(input_size, output_size).to(device)  # Adjust the input size to match the output of the quantum circuit

    def forward(self, x):
        q_features = quantum_net(torch.flatten(x, start_dim=1), self.q_weights_flat)
        q_features_tensor = torch.tensor(q_features, dtype=torch.float32).unsqueeze(0).to(device)
        x = self.fc(q_features_tensor)
        return x

# Assuming the output_size is known
quantum_layer = QuantumNetLayer(input_size=4, output_size=10)  # Adjust the input size to match the output of the quantum circuit


# Replace the fully connected layer in the ResNet model with the QuantumNetLayer
model.fc = quantum_layer




In [16]:
# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [17]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [18]:
# Loss function and optimizer
import torch.optim as optim
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [ ]:
"""
# Training loop
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    # Inside the training loop
    # Inside the training loop
    # Inside the training loop
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        inception_outputs, _ = model(images)

        # Ensure outputs and labels have the same batch size
        labels = labels[:inception_outputs.shape[0]]  # Trim labels if needed to match batch size
        loss = criterion(inception_outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')


        # Adjust learning rate
    # scheduler.step()

"""
# Training loop with dropout
# Training loop
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

print("Training complete.")



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
model.eval()

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {100 * accuracy:.2f}%')


Accuracy on the test set: 9.27%
